In [ ]:
import numpy as np

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [ ]:
with open('archive/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt') as f:  # your file.txt. I had a text file from spermwhale dataset
    raw = f.readlines()

tokens = list()
for line in raw:
    tokens.append(line.lower().replace('\n', '').split()[1:])

In [ ]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)
vocab = list(vocab)

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i


In [ ]:
print(vocab)

In [ ]:
def words2indeces(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

In [ ]:
np.random.seed(1)
embed_size = 10 # заданное кол-во нейронов

embed = (np.random.rand(len(vocab), embed_size) - 0.5) * 0.1 # (матрица весов для каждого слова)

recurrent = np.eye(embed_size) # наша рекуррентная матрица(изначально - единичная)

start = np.zeros(embed_size) # массив нужен для начального этапа прогноза и обучения

decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1 # прогнозирующая матрица

one_hot = np.eye(len(vocab)) # матрица, использующаяся как указание учителя при обратном распространении

In [ ]:
def predict(sent):

    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)
    loss = 0

    for target_i in range(len(sent)):

        layer = {}
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))
        loss += -np.log(layer['pred'][sent[target_i]])
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)
    return layers, loss

In [ ]:
alpha = 0.001

for iter in range(len(tokens) * 10):
    sent = words2indeces(tokens[iter % len(tokens)])
    layers, loss = predict(sent)
    for layer_idx in reversed(range(len(layers))):
   
        layer = layers[layer_idx]
        target = sent[layer_idx - 1]

        if layer_idx > 0:
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            if layer_idx == len(layers) - 1:
                layer['hidden_delta'] = new_hidden_delta 
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx + 1]['hidden_delta'].dot(recurrent.transpose())
        else:
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta'].dot(recurrent.transpose())
    start -= layers[0]['hidden_delta'] * alpha / len(sent)
    for layer_idx, layer in enumerate(layers[1:]):

        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / len(sent)

        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / len(sent)
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / len(sent)
    if iter % 10 == 0:
        print(f'Итерация: {iter}')
        print(f'Perplexity (Затруднение): {str(np.exp(loss / len(sent)))}')